In [ ]:
!pip install pyautogen


In [ ]:
config_list = [
    {
        'model': 'gpt-3.5-turbo-16k',
        'api_key': 'YOUR_API_KEY',
    }
]

In [ ]:
import random
from typing import Optional
import autogen

class TicTacToeAgent(autogen.AssistantAgent):
    def __init__(self, symbol, **kwargs):
        super().__init__(**kwargs)
        self.symbol = symbol  # 'X' or 'O'
        self.board = [[" " for _ in range(3)] for _ in range(3)]
        self.game_over = False
        self.winner = None


    def _check_winner(self):
        # Check rows and columns for a winner
        for i in range(3):
            if self.board[i][0] == self.board[i][1] == self.board[i][2] != " ":
                return self.board[i][0]  # Return the winner's symbol
            if self.board[0][i] == self.board[1][i] == self.board[2][i] != " ":
                return self.board[0][i]  # Return the winner's symbol

        # Check diagonals for a winner
        if self.board[0][0] == self.board[1][1] == self.board[2][2] != " " or \
           self.board[0][2] == self.board[1][1] == self.board[2][0] != " ":
            return self.board[1][1]  # Return the winner's symbol

        # Check for a tie (if the board is full and no winner)
        if all(self.board[i][j] != " " for i in range(3) for j in range(3)):
            return "TIE"  # Return a special marker for a tie

        # If no winner and not a tie, game continues
        return None

    def _make_move(self, row, col, symbol):
        if self.board[row][col] == " ":
            self.board[row][col] = symbol
            result = self._check_winner()
            if result is not None:
                self.game_over = True
                if result == "TIE":
                    self.winner = None  # No winner in case of a tie
                else:
                    self.winner = result  # Set the winner
                return True
            return True  # Move made, but game continues
        return False  # Move could not be made, cell already occupied


    def _generate_reply(self, messages: Optional[list] = None, sender: Optional[autogen.Agent] = None, config: Optional[any] = None) -> str:
        # Game over scenario
        if self.game_over:
            goodbye_message = "Goodbye and thanks for the game!" if self.name == "Carl" else ""
            return f"Game over! Winner: {self.winner}" + goodbye_message if self.winner else "Game over! It's a tie." + goodbye_message

        conversation_reply = ""
        if messages:
            # Check if the last message is about the football match
            if "football" in messages[-1]['content'].lower():
                conversation_reply = "That was an exciting match yesterday! But let's focus on our game now."

            try:
                # Attempt to parse the last message for a move
                content = messages[-1]['content']
                if 'row' in content and 'col' in content:  # This is a simplified check
                    row, col = map(int, content.split()[-2:])  # Assuming the last two numbers are row and col
                    if 0 <= row <= 2 and 0 <= col <= 2:  # Validate row and column
                        if self._make_move(row, col, self.symbol):
                            return f"{conversation_reply} Placed {self.symbol} at ({row}, {col})."
                        else:
                            return f"{conversation_reply} That spot is taken, try another."
                    else:
                        return f"{conversation_reply} Move out of bounds. Please try again."
            except ValueError:
                # Handle invalid message format for moves, assuming other content could be conversational
                return "Invalid move format. Please send moves in 'row col' format. Or were you talking about the football match?"

        # If no specific messages or only conversational messages are provided, and it's this agent's turn
        if not any(self.board[i][j] != " " for i in range(3) for j in range(3)):
            self.game_over = True
            return "Game over! It's a tie."  # This might need revising if it could be a win condition instead

        # Randomly place a move if no valid move or conversation is provided
        while True:
            row, col = random.randint(0, 2), random.randint(0, 2)
            if self._make_move(row, col, self.symbol):
                return f"{conversation_reply} Starting the game, placed {self.symbol} at ({row}, {col})."

        # If the method hasn't returned yet, add a generic reply to keep the conversation or game going
        return "Let's keep playing!" if not conversation_reply else conversation_reply





player2 = TicTacToeAgent('X', name="Carl", system_message="I'm Player X, ready to win!I'll not mark X in a cell thats already taken by the opponent player. I'll have a casual chat with the opponent player about the football match I saw yesterday while playing the game.I'll stop the conversation once the game ends with a goodbye", llm_config={"temperature": 0.5, "config_list": config_list})
player1 = TicTacToeAgent('O', name="John", system_message="Player O here, let's see who wins!.I'll not mark O in a cell thats already taken.  I'll stop replying to my opponent player while playing the game.I'll end replying once the opponenet says goodbye", llm_config={"temperature": 0.5, "config_list": config_list})

# Example move, replace with your game loop or interaction logic
player2.initiate_chat(player1, message="Let's Start the game")  # Player 1 starts by placing 'X' at top-left corner


Carl (to John):

Let's Start the game

--------------------------------------------------------------------------------
John (to Carl):

Great! Let's start the game. Here's the initial game board:

1 | 2 | 3
---------
4 | 5 | 6
---------
7 | 8 | 9

It's your turn. Please choose a number from 1 to 9 to place your X.

--------------------------------------------------------------------------------
Carl (to John):

Thanks! I'll choose number 5 to place my X.

By the way, did you happen to catch the football match yesterday? It was quite an intense game!

--------------------------------------------------------------------------------
John (to Carl):

Great choice! You've placed your X in the center square. Here's the updated game board:

1 | 2 | 3
---------
4 | X | 6
---------
7 | 8 | 9

I'm sorry, but I'm an AI and I don't have the ability to watch or follow sports events. However, I can definitely understand the excitement that comes with intense games. Now it's my turn.

I'll choose nu

KeyboardInterrupt: 